In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
import threading
from Utility import spawn_camera, preprocess_image, riconosci_parcheggio, control_retro, controllo_velocita, find_point
from ParkingIdentifier import process_image

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [3]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-1, -25, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-0.8, y=0.5, z=1.9), carla.Rotation(yaw=90, pitch=-40)))
run = True

In [4]:
radar_bp = world.get_blueprint_library().find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '10')  # Horizontal field of view
radar_bp.set_attribute('vertical_fov', '10')    # Vertical field of view
radar_bp.set_attribute('range', '20')           # Maximum range
radar_transform = carla.Transform(carla.Location(x=2.0, z=1.0))
radar = world.spawn_actor(radar_bp, radar_transform, attach_to=vehicle)

In [5]:
"""
def camera_rigth_callback(image):
    global run
    parkingfound,corners=process_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))
    if parkingfound:
        run = False

def thread_function():
    while run:
        camera_rigth.listen(lambda image: camera_rigth_callback(image))

"""
def camera_rigth_callback(image):
    global run
    parkingfound,corners=process_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))
    if parkingfound:
        run = False

min_ttc = float('inf')
def radar_callback(data: carla.RadarMeasurement):
    global min_ttc, min_distance
    min_ttc = float('inf')
    for detection, i in zip(data, range(len(data))):
        absolute_speed = abs(detection.velocity)
        # Calculate TTC
        if absolute_speed != 0:
            ttc = detection.depth / absolute_speed
            if ttc < min_ttc:
                min_ttc = ttc

# Register the radar callback
radar.listen(radar_callback)

In [6]:
target_speed_mps = 10 / 3.6
camera_rigth.listen(lambda image: camera_rigth_callback(image))

control = carla.VehicleControl()
control.steer = 0.0
control.brake = 0.0
target_distance = 1.1
distance_travelled = 0.0

n_listem_break=0
ttc_threshold = 1.0
collision = False

while run:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    control = controllo_velocita(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
    if min_ttc < ttc_threshold:
            control = carla.VehicleControl()
            control.brake = 1.0  
            vehicle.apply_control(control)
            print("Emergency braking activated!")
            n_listem_break += 1
            if n_listem_break>10:
                collision = True
                break
    time.sleep(0.05)
while distance_travelled < target_distance and not collision:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    distance_travelled += current_speed_mps * 0.05
    control = controllo_velocita(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
    time.sleep(0.05)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
control.brake = 0.0
control.reverse = True
control.steer = 0.8
distance_travelled = 0.0
target_distance = 3.2
while distance_travelled < target_distance and not collision:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    distance_travelled += current_speed_mps * 0.05
    print(control)
    control = controllo_velocita(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
    time.sleep(0.05)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
camera_rigth.destroy()
image1 = None
def camera_callback2(image):
    global image1
    image1 =  np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
camera_retro.listen(lambda image: camera_callback2(image))
time.sleep(0.15)
while True:
    if control_retro(vehicle, find_point(image1), control):
        break
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(4.1)
camera_retro.destroy()
vehicle.destroy()

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'

...

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'

AttributeError: 'numpy.ndarray' object has no attribute 'save_to_disk'